# Markov Chain Lab
(Derived from CEE599_MarkovChain_Lab.m, Jessica Lundquist, Oct 23, 2015)

In [1]:
import numpy as np
import scipy.stats as st
from scipy import sparse
import matplotlib.pyplot as plt

%matplotlib inline

****
#### City and Suburbs:
Let's define two states:
 - city = 0
 - suburbs = 1

(see Weather Example in Lecture notes, except note that we now have an initial condition, which would be like just saying today's weather was dry) 

Set up the Markov probability matrix:

In [2]:
Pmarkov = np.zeros([2,2]) # create an empty 2-by-2 numpy array

Given someone lives in the city, they have 95% chance of staying and 5% chance of moving:

In [3]:
# Assign our given values to the table:
Pmarkov[0,0] = 0.95 # note that we use the array indices to describe probability of going from state 0 to state 0
Pmarkov[0,1] = 0.05 # probability of going from state 0 to state 1

Given someone lives in the suburbs, they have a 97% change of staying and a 3% chance of moving:

In [4]:
# Assign our given values to the table:
Pmarkov[1,0] = 0.03 # probability of going from state 1 to state 0
Pmarkov[1,1] = 0.97 # probability of going from state 1 to state 1

View the complete matrix:

In [5]:
print(Pmarkov)

[[0.95 0.05]
 [0.03 0.97]]


Before going into matrix notation, let's think this through. In the initial state, 60% of people live in the city & 40% live in the suburb.

In [6]:
pcity0 = 0.6
psuburb0 = 0.4

Take one step through the Markov chain. What fraction of people live in the city after one step?

In [7]:
pcity1 = Pmarkov[0,0]*pcity0 + Pmarkov[1,0]*psuburb0
print(pcity1)

0.582


What fraction of people live in the suburbs?

In [8]:
psuburb1 = Pmarkov[0,1]*pcity0 + Pmarkov[1,1]*psuburb0
print(psuburb1)

0.41800000000000004


Draw a picture of what you are doing in terms of matrix multiplication.

Now use matrix algebra using numpy to get the same answer and check your work.

In [9]:
# initial state of populations for state 0 and state 1
x0 = np.array([pcity0, psuburb0])

# multiply initial states with Markov matrix
x1 = np.dot(x0, Pmarkov)

# check values of x1 here against what you got for pcity1 and pusuburb1
print(x1)

[0.582 0.418]


Now we can make this the current state and take a second step with the matrix (using both one step and matrix notation to check our work):

In [10]:
# take a second step with the Markov matrix:
pcity2 = Pmarkov[0,0]*pcity1 + Pmarkov[1,0]*psuburb1
psuburb2=Pmarkov[0,1]*pcity1 + Pmarkov[1,1]*psuburb1
print(pcity2,psuburb2)

# take a second step with the Markov matrix (using np.dot)
x2 = np.dot(x1,Pmarkov)
print(x2[0],x2[1])


0.5654399999999999 0.43456000000000006
0.5654399999999999 0.43455999999999995


We can also make multiple steps by raising the Markov matrix to a power:

In [11]:
# three steps
x3 = np.dot(x0, np.linalg.matrix_power(Pmarkov,3))
x3_check = np.dot(x2,Pmarkov)
print(x3)
print(x3_check)

[0.5502048 0.4497952]
[0.5502048 0.4497952]


In [12]:
# 10 steps, then 100 steps:
x10 = np.dot(x0, np.linalg.matrix_power(Pmarkov,10))
x100 = np.dot(x0, np.linalg.matrix_power(Pmarkov,100))
print(x10)
print(x100)

[0.4727374 0.5272626]
[0.37505382 0.62494618]


And now, use the eigenvector function in numpy to see if x100 is the same as the steady state matrix:

In [13]:
[W, V] = np.linalg.eig(Pmarkov.T)

In [14]:
V

array([[-0.70710678, -0.51449576],
       [ 0.70710678, -0.85749293]])

Find the column of V associated with the eigenvalue of 1, but allow for some margin of round-off error

(from the function documentation: the column ```V[:,i]``` is the eigenvector corresponding to the eigenvalue ```W[i]```)

In [15]:
# First, we can inspect W by eye (since this is a small matrix)
W

array([0.92, 1.  ])

In [16]:
# We can also tell the computer to find the value of W clost to 1 (within round-off error)
# and then look at the associated column of V
err = 0.000000001
col1 = np.where((W>=1-err) & (W<=1+err))
p = V[:,col1]
p = p/np.sum(p) # normalize the eigenvector so that they sum to one

print('Eigenvector Steady State Probabilities: {}'.format(p.T[0][0]))
# Yes, these are very close to the values we got for x100
print('Markov Chain 100-step Probabilities: {}'.format(x100))


Eigenvector Steady State Probabilities: [0.375 0.625]
Markov Chain 100-step Probabilities: [0.37505382 0.62494618]


***

#### Now given a timeseries of four states, determine the Markov Matrix and use it to generate a timeseries

In [17]:
# first, read in data that I generated (sequence of states 1, 2, 3, and 4):
data = np.genfromtxt('markov_random4.txt',dtype=int)

In [18]:
# This counts the transitions from each state to the next and marks that count
S = sparse.csr_matrix((np.ones_like(data[:-1]), (data[:-1], data[1:])), dtype=np.float)

# This converts those counts to matrix form
tm = S.todense()
print(tm)

[[ 0.  0.  0.  0.  0.]
 [ 0. 16. 53. 22. 28.]
 [ 0. 31. 34. 12. 56.]
 [ 0. 13. 34. 48. 28.]
 [ 0. 58. 12. 41. 13.]]


#### Take a look at the matrix above, what does this represent?
We had 16 counts when the series transitioned from 1 to 1, 53 counts when it transitioned from 1 to 2, etc.
We want to transition these from counts to frequencies.
To do this, we need to normalize the transition matrix to get probabilities

In [23]:
tm_norm = np.zeros_like(tm)
for i in range(0,tm.shape[0]):
    tm_norm[i,:] = tm[i,:] / np.sum(tm[i,:])
    
print(tm_norm) # This is our normalized transition matrix.

[[       nan        nan        nan        nan        nan]
 [0.         0.13445378 0.44537815 0.18487395 0.23529412]
 [0.         0.23308271 0.2556391  0.09022556 0.42105263]
 [0.         0.10569106 0.27642276 0.3902439  0.22764228]
 [0.         0.46774194 0.09677419 0.33064516 0.10483871]]


C:\Users\steve\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [20]:
# We take the above probabilities of transitions, and turn them into discrete CDF's.
# These will allow us to map random numbers generated from a uniform distribution into 
# transitions that follow these probability rules.
tm_cdf = np.cumsum(tm_norm,1)

Now, we want a "random walk" for 2000 years that follows these transition probabilities.

In [21]:
n_years = 2000
q = np.random.uniform(0,1,n_years); # uniformly distributed random numbers n_years long

initialstate = 2; # give it an initial state, doesn't really matter which

Nrand = np.zeros_like(q) # initialize an array of the proper size, with the initial state
Nrand[0] = initialstate;

# Now, just like we did when we created monte carlo simulations from empirical CDFs,
# we use our uniform random numbers to look up the next state in the transition matrix
for i in range(1,n_years):
    if q[i] <= tm_cdf[int(Nrand[i-1]),1]: #probability of transitioning from state i to 1
        Nrand[i] = 1;
    elif q[i] <= tm_cdf[int(Nrand[i-1]),2]: #transition from state i to 2
        Nrand[i] = 2;
    elif q[i] <= tm_cdf[int(Nrand[i-1]),3]: #transition from state i to 3
        Nrand[i] = 3;
    else:
        Nrand[i] = 4;

In [22]:
# And how many times did state 3 appear 4 times?
Test3 = [Nrand[0:-3], Nrand[1:-2], Nrand[2:-1], Nrand[3:]] # stack our data 4 times, shifting it to the right by 1 each time
Test3 = np.stack(Test3, axis=1)

G2 = np.where((np.max(Test3, axis=1) == 3) & (np.min(Test3, axis=1) == 3))
# if both the maximum and the minimum are 3, then we have 4 threes in our sequence

frequencyof4threes = G2[0].size / Test3.shape[0]

print('Frequency of four 3s in a row = {}%'.format(np.round(frequencyof4threes,3)))

Frequency of four 3s in a row = 0.017%
